# 4.8 Task

### Step 1

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create Path
path = r'C:\Users\jeffr\Documents\Instacart Basket Analysis'

In [3]:
# Importing ords_prods_merge2 data set
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge2.pkl'))

In [5]:
# Confirming Import
ords_prods_merge.shape

(32404859, 24)

### Step 2

In [6]:
# find aggregated mean of order_number column grouped by department_id
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### Step 3

In [7]:
# I believe because my daa set isn't sorted the way the one used in the Exercise is, all department_ids showed up just like they did when I went through the Exercise using a subset

### Step 4

In [8]:
# Creating loyalty flag for existing customers
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\jeffr\AppData\Local\Temp\ipykernel_42448\788592546.py:2: FutureWarning: The provided callable <function max at 0x000001A430990400> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [9]:
# Confirming max_order column created
ords_prods_merge.head(15)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,eval_set,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,...,aisle_id,department_id,prices,match,price_range_loc,prices_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,0,0,2539329,1,prior,1,2,8,NaN,196,...,77,7,9.0,both,Mid-range product,NaN,Regularly busy,Regularly Busy,Average Orders,10
1,0,0,2539329,1,prior,1,2,8,NaN,14084,...,91,16,12.5,both,Mid-range product,NaN,Regularly busy,Regularly Busy,Average Orders,10
2,0,0,2539329,1,prior,1,2,8,NaN,12427,...,23,19,4.4,both,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10
3,0,0,2539329,1,prior,1,2,8,NaN,26088,...,23,19,4.7,both,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10
4,0,0,2539329,1,prior,1,2,8,NaN,26405,...,54,17,1.0,both,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10
5,1,1,2398795,1,prior,2,3,7,15.0,196,...,77,7,9.0,both,Mid-range product,NaN,Regularly busy,Slowest Days,Average Orders,10
6,1,1,2398795,1,prior,2,3,7,15.0,10258,...,117,19,3.0,both,NaN,Low-range product,Regularly busy,Slowest Days,Average Orders,10
7,1,1,2398795,1,prior,2,3,7,15.0,12427,...,23,19,4.4,both,NaN,Low-range product,Regularly busy,Slowest Days,Average Orders,10
8,1,1,2398795,1,prior,2,3,7,15.0,13176,...,24,4,10.3,both,Mid-range product,NaN,Regularly busy,Slowest Days,Average Orders,10
9,1,1,2398795,1,prior,2,3,7,15.0,26088,...,23,19,4.7,both,NaN,Low-range product,Regularly busy,Slowest Days,Average Orders,10


In [11]:
# Create flag that assigns loyalty label to user ID on its max order value
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regualr Customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [12]:
# Check value counts
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regualr Customer    15876776
Loyal Customer      10284093
New Customer         6243990
Name: count, dtype: int64

### Step 5

In [13]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'max', 'min']})

prices              
                       mean      max  min
loyalty_flag                             
Loyal Customer    10.386336  99999.0  1.0
New Customer      13.294670  99999.0  1.0
Regualr Customer  12.495717  99999.0  1.0

In [14]:
# It appears on average, the loyal Customers purchase lower priced items while New Customers purchase items with higher prices. It appears the while Loyal Customers place more orders, they tend to purchase less expensive items and vice vera for New Customers

### Step 6

In [16]:
# Create spending flag for each user based on average price of all their orders
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\jeffr\AppData\Local\Temp\ipykernel_42448\4030992836.py:2: FutureWarning: The provided callable <function mean at 0x000001A430990E00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [17]:
# Confirming average_price column created
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,eval_set,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,...,prices,match,price_range_loc,prices_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price
0,0,0,2539329,1,prior,1,2,8,NaN,196,...,9.0,both,Mid-range product,NaN,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797
1,0,0,2539329,1,prior,1,2,8,NaN,14084,...,12.5,both,Mid-range product,NaN,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797
2,0,0,2539329,1,prior,1,2,8,NaN,12427,...,4.4,both,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797
3,0,0,2539329,1,prior,1,2,8,NaN,26088,...,4.7,both,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797
4,0,0,2539329,1,prior,1,2,8,NaN,26405,...,1.0,both,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797


In [18]:
# Creating spending flag based on average_price
ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'spending_flag'] = 'Low Spender'
ords_prods_merge.loc[ords_prods_merge['average_price'] >= 10, 'spending_flag'] = 'High Spender'

In [19]:
# Check value counts
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

### Step 7

In [20]:
# Creating frequency flag based on user ordering behavior
ords_prods_merge['median_days_since_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\jeffr\AppData\Local\Temp\ipykernel_42448\236046594.py:2: FutureWarning: The provided callable <function median at 0x000001A430ABBA60> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_since_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [21]:
# Confirm median_days_since_orders column
ords_prods_merge.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,eval_set,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,...,price_range_loc,prices_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days_since_orders
0,0,0,2539329,1,prior,1,2,8,NaN,196,...,Mid-range product,NaN,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
1,0,0,2539329,1,prior,1,2,8,NaN,14084,...,Mid-range product,NaN,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
2,0,0,2539329,1,prior,1,2,8,NaN,12427,...,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
3,0,0,2539329,1,prior,1,2,8,NaN,26088,...,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5
4,0,0,2539329,1,prior,1,2,8,NaN,26405,...,NaN,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5


In [22]:
# Create frequency flags based on the median days between orders
ords_prods_merge.loc[ords_prods_merge['median_days_since_orders'] > 20, 'frequency_flag'] = 'Non-Frequent Customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_orders'] > 10) & (ords_prods_merge['median_days_since_orders'] <= 20), 'spending_flag'] = 'Regular Customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_orders'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [23]:
# Check Value Counts
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent Customer        21559853
NaN                       7208569
Non-Frequent Customer     3636437
Name: count, dtype: int64

In [24]:
# Check Columns
ords_prods_merge.head(10)

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,eval_set,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,...,prices_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_days_since_orders,frequency_flag
0,0,0,2539329,1,prior,1,2,8,NaN,196,...,NaN,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
1,0,0,2539329,1,prior,1,2,8,NaN,14084,...,NaN,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
2,0,0,2539329,1,prior,1,2,8,NaN,12427,...,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
3,0,0,2539329,1,prior,1,2,8,NaN,26088,...,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
4,0,0,2539329,1,prior,1,2,8,NaN,26405,...,Low-range product,Regularly busy,Regularly Busy,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
5,1,1,2398795,1,prior,2,3,7,15.0,196,...,NaN,Regularly busy,Slowest Days,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
6,1,1,2398795,1,prior,2,3,7,15.0,10258,...,Low-range product,Regularly busy,Slowest Days,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
7,1,1,2398795,1,prior,2,3,7,15.0,12427,...,Low-range product,Regularly busy,Slowest Days,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
8,1,1,2398795,1,prior,2,3,7,15.0,13176,...,NaN,Regularly busy,Slowest Days,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer
9,1,1,2398795,1,prior,2,3,7,15.0,26088,...,Low-range product,Regularly busy,Slowest Days,Average Orders,10,New Customer,6.367797,Low Spender,20.5,Non-Frequent Customer


### Steps 8 and 9

In [25]:
# Exporting dataframe to pkl
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge3.pkl'))